# Importing the Data

- Import relevant imdb files for movie/actor data

In [1]:
import pandas as pd

In [2]:
#name of people on titles. known_for_titles matches the title_id
imdb_name = pd.read_csv('zippedData/imdb.name.basics.csv.gz') 

#title, region, language
imdb_title_a =  pd.read_csv('zippedData/imdb.title.akas.csv.gz') 

#title, year, runtime, genre
imdb_title_b =  pd.read_csv('zippedData/imdb.title.basics.csv.gz') 

#categorizes the people(nameid) on the movie(titleid) and includes their job role and characters they played
imdb_title_p = pd.read_csv('zippedData/imdb.title.principals.csv.gz') 

# Movie Data
- Merge IMDB dataframes related to just movies
- Specify movies released in the US
- Drop unnecessary columns and duplicate values
- Import movie data from API calls
- Create functions to pull out values from nested data
- Create new columns for genre, season, rating, and profitability
- Remove null and 0 values

In [3]:
#renamed the 'tconst' column to match the title_id column from imdb_title_a for merging 
imdb_title_b = imdb_title_b.rename(columns = {"tconst": "title_id"})


In [4]:
#merge b and a df's first to just get all of the title info and the region in one table

title_ba = imdb_title_b.merge(imdb_title_a, on = 'title_id', how = 'left')
title_ba.head()

,title_id,primary_title,original_title,start_year,runtime_minutes,genres,ordering,title,region,language,types,attributes,is_original_title
0,tt0063540,Sunghursh,Sunghursh,2013,175.0,"Action,Crime,Drama",1.0,Sangharsh,IN,hi,NaN,alternative transliteration,0.0
1,tt0063540,Sunghursh,Sunghursh,2013,175.0,"Action,Crime,Drama",2.0,Sunghursh,NaN,NaN,original,NaN,1.0
2,tt0063540,Sunghursh,Sunghursh,2013,175.0,"Action,Crime,Drama",3.0,Sunghursh,IN,NaN,NaN,NaN,0.0
3,tt0063540,Sunghursh,Sunghursh,2013,175.0,"Action,Crime,Drama",4.0,Sunghursh,IN,hi,NaN,alternative transliteration,0.0
4,tt0063540,Sunghursh,Sunghursh,2013,175.0,"Action,Crime,Drama",5.0,Sungharsh,IN,hi,NaN,alternative spelling,0.0


In [5]:
#get US movies
us_titles = title_ba.groupby('region').get_group('US')
us_titles.head()

,title_id,primary_title,original_title,start_year,runtime_minutes,genres,ordering,title,region,language,types,attributes,is_original_title
14,tt0069049,The Other Side of the Wind,The Other Side of the Wind,2018,122.0,Drama,2.0,The Other Side of the Wind,US,NaN,imdbDisplay,NaN,0.0
31,tt0112502,Bigfoot,Bigfoot,2017,NaN,"Horror,Thriller",1.0,Bigfoot,US,NaN,NaN,NaN,0.0
52,tt0159369,Cooper and Hemingway: The True Gen,Cooper and Hemingway: The True Gen,2013,180.0,Documentary,3.0,Cooper and Hemingway: The True Gen,US,NaN,NaN,NaN,0.0
63,tt0192528,Heaven & Hell,Reverse Heaven,2018,104.0,Drama,1.0,Heaven & Hell,US,NaN,imdbDisplay,NaN,0.0
65,tt0230212,The Final Journey,The Final Journey,2010,120.0,Drama,1.0,The Final Journey,US,NaN,NaN,NaN,0.0


In [6]:
#drop any columns we don't need
clean_us_movies = us_titles.drop(columns = ['primary_title', 'original_title', 'runtime_minutes', 'start_year','language', 'types', 'attributes', 'is_original_title'])
#drop duplicates that have the same title_id AND title
clean_us_movies = clean_us_movies.drop_duplicates(subset=(['title_id', 'title']), keep="first")

In [7]:
#drop any remaining duplicates that just have the same title_id
clean_us_movies = clean_us_movies.drop_duplicates(subset=('title_id'), keep="first")

In [8]:
#export clean_us_movies to use title_id's for API calls

#clean_us_movies.to_csv('clean_us_moves.csv', index=False)

In [9]:
#import us movie data df that now includes API movie data

#budget, genres, release date, revenue, title, ratings
movie_data = pd.read_csv('./movie_data.csv')

In [10]:
#drop columns from movie_data
movie_data = movie_data.drop(columns = ['Unnamed: 0', 'original_language', 'original_title', 'production_companies'])
movie_data.head()

,budget,genres,id,imdb_id,release_date,revenue,title,results
0,12000000.0,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",299782.0,tt0069049,2018-11-02,0.0,The Other Side of the Wind,"[{'iso_3166_1': 'US', 'release_dates': [{'cert..."
1,0.0,"[{'id': 99, 'name': 'Documentary'}]",224332.0,tt0159369,2013-09-27,0.0,Cooper and Hemingway: The True Gen,"[{'iso_3166_1': 'US', 'release_dates': [{'cert..."
2,5000000.0,"[{'id': 18, 'name': 'Drama'}]",567662.0,tt0192528,2018-11-06,0.0,Heaven & Hell,"[{'iso_3166_1': 'US', 'release_dates': [{'cert..."
3,65000000.0,"[{'id': 16, 'name': 'Animation'}, {'id': 28, '...",116977.0,tt0249516,2012-06-15,73706.0,Foodfight!,"[{'iso_3166_1': 'US', 'release_dates': [{'cert..."
4,2000000.0,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",323498.0,tt0255820,2013-08-11,0.0,Return to Babylon,"[{'iso_3166_1': 'US', 'release_dates': [{'cert..."


In [11]:
#convert genres string to a list for later manipulation
for name in range(len(movie_data.genres)):
    movie_data.genres[name] = eval(movie_data.genres[name])
movie_data.head()

/Users/danielmocombe/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,budget,genres,id,imdb_id,release_date,revenue,title,results
0,12000000.0,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",299782.0,tt0069049,2018-11-02,0.0,The Other Side of the Wind,"[{'iso_3166_1': 'US', 'release_dates': [{'cert..."
1,0.0,"[{'id': 99, 'name': 'Documentary'}]",224332.0,tt0159369,2013-09-27,0.0,Cooper and Hemingway: The True Gen,"[{'iso_3166_1': 'US', 'release_dates': [{'cert..."
2,5000000.0,"[{'id': 18, 'name': 'Drama'}]",567662.0,tt0192528,2018-11-06,0.0,Heaven & Hell,"[{'iso_3166_1': 'US', 'release_dates': [{'cert..."
3,65000000.0,"[{'id': 16, 'name': 'Animation'}, {'id': 28, '...",116977.0,tt0249516,2012-06-15,73706.0,Foodfight!,"[{'iso_3166_1': 'US', 'release_dates': [{'cert..."
4,2000000.0,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",323498.0,tt0255820,2013-08-11,0.0,Return to Babylon,"[{'iso_3166_1': 'US', 'release_dates': [{'cert..."


In [12]:
#convert results from string to list for later manipulation
for name in range(len(movie_data.results)):
    movie_data.results[name] = eval(movie_data.results[name])
movie_data.head()

/Users/danielmocombe/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,budget,genres,id,imdb_id,release_date,revenue,title,results
0,12000000.0,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",299782.0,tt0069049,2018-11-02,0.0,The Other Side of the Wind,"[{'iso_3166_1': 'US', 'release_dates': [{'cert..."
1,0.0,"[{'id': 99, 'name': 'Documentary'}]",224332.0,tt0159369,2013-09-27,0.0,Cooper and Hemingway: The True Gen,"[{'iso_3166_1': 'US', 'release_dates': [{'cert..."
2,5000000.0,"[{'id': 18, 'name': 'Drama'}]",567662.0,tt0192528,2018-11-06,0.0,Heaven & Hell,"[{'iso_3166_1': 'US', 'release_dates': [{'cert..."
3,65000000.0,"[{'id': 16, 'name': 'Animation'}, {'id': 28, '...",116977.0,tt0249516,2012-06-15,73706.0,Foodfight!,"[{'iso_3166_1': 'US', 'release_dates': [{'cert..."
4,2000000.0,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",323498.0,tt0255820,2013-08-11,0.0,Return to Babylon,"[{'iso_3166_1': 'US', 'release_dates': [{'cert..."


In [13]:
#convert release date from string to datetime type
movie_data['release_date'] = pd.to_datetime(movie_data['release_date'])


In [14]:
#create a column for the month
movie_data['month'] = pd.DatetimeIndex(movie_data['release_date']).month


In [15]:
#create a column for the year
movie_data['year'] = pd.DatetimeIndex(movie_data['release_date']).year
movie_data.head()

,budget,genres,id,imdb_id,release_date,revenue,title,results,month,year
0,12000000.0,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",299782.0,tt0069049,2018-11-02,0.0,The Other Side of the Wind,"[{'iso_3166_1': 'US', 'release_dates': [{'cert...",11,2018
1,0.0,"[{'id': 99, 'name': 'Documentary'}]",224332.0,tt0159369,2013-09-27,0.0,Cooper and Hemingway: The True Gen,"[{'iso_3166_1': 'US', 'release_dates': [{'cert...",9,2013
2,5000000.0,"[{'id': 18, 'name': 'Drama'}]",567662.0,tt0192528,2018-11-06,0.0,Heaven & Hell,"[{'iso_3166_1': 'US', 'release_dates': [{'cert...",11,2018
3,65000000.0,"[{'id': 16, 'name': 'Animation'}, {'id': 28, '...",116977.0,tt0249516,2012-06-15,73706.0,Foodfight!,"[{'iso_3166_1': 'US', 'release_dates': [{'cert...",6,2012
4,2000000.0,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",323498.0,tt0255820,2013-08-11,0.0,Return to Babylon,"[{'iso_3166_1': 'US', 'release_dates': [{'cert...",8,2013


In [16]:
#pulls out just the genre names and puts them into a single list
for x in range(len(movie_data.genres)):
    movie_data.genres[x] = [y['name'] for y in movie_data.genres[x]]


/Users/danielmocombe/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [17]:
#function that will get the value for certification key which is a nested list of dictionaries under results column
def get_cert(test):
    for x in range(len(test)):
        for y in test[x].keys():
            while test[x][y] == 'US':
                return test[x]['release_dates'][0]['certification']
            continue

In [18]:
#function that shows the name of the seasion for each month based on Box Office Mojo's categorization
def season(x):
    if x >= 1 and x <= 2:
        return 'Winter'
    elif x >= 3 and x <= 4:
        return 'Spring'
    elif x >= 5 and x <= 8:
        return 'Summer'
    elif x >= 9 and x <= 10:
        return 'Fall' 
    else:
        return 'Holiday'

In [19]:
#create new column that applies get_cert function to the df
movie_data['rating'] = movie_data.results.apply(get_cert)
#create new column that applies season function to the df
movie_data['season'] = movie_data.month.apply(season)
#create column that just shows first genre from genres list
movie_data['genre'] = movie_data['genres'].str[0]
#create column that calculates the profitability per movie
movie_data['profitability'] = movie_data['revenue'] - movie_data['budget']


In [20]:
#remove null rating values
clean_movie_data = movie_data.dropna(subset=['rating'])

#remove movies with 0 budgets
clean_movie_data = clean_movie_data.loc[~(clean_movie_data['budget'] == 0)]

#remove null genre values
clean_movie_data = clean_movie_data.dropna(subset=['rating'])


In [21]:
#sort results by revenue
clean_movie_data = clean_movie_data.sort_values(by=['revenue'], ascending=[False])
clean_movie_data.head(10)

,budget,genres,id,imdb_id,release_date,revenue,title,results,month,year,rating,season,genre,profitability
15167,356000000.0,"[Adventure, Science Fiction, Action]",299534.0,tt4154796,2019-04-24,2.797801e+09,Avengers: Endgame,"[{'iso_3166_1': 'IT', 'release_dates': [{'cert...",4,2019,PG-13,Spring,Adventure,2.441801e+09
9384,245000000.0,"[Action, Adventure, Science Fiction, Fantasy]",140607.0,tt2488496,2015-12-15,2.068224e+09,Star Wars: The Force Awakens,"[{'iso_3166_1': 'IT', 'release_dates': [{'cert...",12,2015,,Holiday,Action,1.823224e+09
15166,300000000.0,"[Adventure, Action, Science Fiction]",299536.0,tt4154756,2018-04-25,2.046240e+09,Avengers: Infinity War,"[{'iso_3166_1': 'US', 'release_dates': [{'cert...",4,2018,PG-13,Spring,Adventure,1.746240e+09
18,150000000.0,"[Action, Adventure, Science Fiction, Thriller]",135397.0,tt0369610,2015-06-06,1.671713e+09,Jurassic World,"[{'iso_3166_1': 'GR', 'release_dates': [{'cert...",6,2015,PG-13,Summer,Action,1.521713e+09
19625,260000000.0,"[Adventure, Family]",420818.0,tt6105098,2019-07-12,1.656943e+09,The Lion King,"[{'iso_3166_1': 'KR', 'release_dates': [{'cert...",7,2019,PG,Summer,Adventure,1.396943e+09
210,220000000.0,"[Science Fiction, Action, Adventure]",24428.0,tt0848228,2012-04-25,1.519558e+09,The Avengers,"[{'iso_3166_1': 'GR', 'release_dates': [{'cert...",4,2012,PG-13,Spring,Science Fiction,1.299558e+09
10542,190000000.0,"[Action, Thriller]",168259.0,tt2820852,2015-04-01,1.515048e+09,Furious 7,"[{'iso_3166_1': 'US', 'release_dates': [{'cert...",4,2015,PG-13,Spring,Action,1.325048e+09
8842,250000000.0,"[Action, Adventure, Science Fiction]",99861.0,tt2395427,2015-04-22,1.405404e+09,Avengers: Age of Ultron,"[{'iso_3166_1': 'US', 'release_dates': [{'cert...",4,2015,PG-13,Spring,Action,1.155404e+09
4647,200000000.0,"[Action, Adventure, Fantasy, Science Fiction]",284054.0,tt1825683,2018-02-13,1.346739e+09,Black Panther,"[{'iso_3166_1': 'US', 'release_dates': [{'cert...",2,2018,PG-13,Winter,Action,1.146739e+09
662,125000000.0,"[Fantasy, Adventure]",12445.0,tt1201607,2011-07-07,1.341511e+09,Harry Potter and the Deathly Hallows: Part 2,"[{'iso_3166_1': 'KR', 'release_dates': [{'cert...",7,2011,PG-13,Summer,Fantasy,1.216511e+09


In [22]:
#drop any remaining columns we don't need for analysis

clean_movie_data = clean_movie_data.drop(columns = ['genres', 'results', 'id'])

#only keep rows that have an mpaa rating
clean_movie_data = clean_movie_data[clean_movie_data['rating'].isin(['G', 'PG', 'PG-13', 'R'])]

In [23]:
clean_movie_data.head(50)

,budget,imdb_id,release_date,revenue,title,month,year,rating,season,genre,profitability
15167,356000000.0,tt4154796,2019-04-24,2.797801e+09,Avengers: Endgame,4,2019,PG-13,Spring,Adventure,2.441801e+09
15166,300000000.0,tt4154756,2018-04-25,2.046240e+09,Avengers: Infinity War,4,2018,PG-13,Spring,Adventure,1.746240e+09
18,150000000.0,tt0369610,2015-06-06,1.671713e+09,Jurassic World,6,2015,PG-13,Summer,Action,1.521713e+09
19625,260000000.0,tt6105098,2019-07-12,1.656943e+09,The Lion King,7,2019,PG,Summer,Adventure,1.396943e+09
210,220000000.0,tt0848228,2012-04-25,1.519558e+09,The Avengers,4,2012,PG-13,Spring,Science Fiction,1.299558e+09
10542,190000000.0,tt2820852,2015-04-01,1.515048e+09,Furious 7,4,2015,PG-13,Spring,Action,1.325048e+09
8842,250000000.0,tt2395427,2015-04-22,1.405404e+09,Avengers: Age of Ultron,4,2015,PG-13,Spring,Action,1.155404e+09
4647,200000000.0,tt1825683,2018-02-13,1.346739e+09,Black Panther,2,2018,PG-13,Winter,Action,1.146739e+09
662,125000000.0,tt1201607,2011-07-07,1.341511e+09,Harry Potter and the Deathly Hallows: Part 2,7,2011,PG-13,Summer,Fantasy,1.216511e+09
9554,200000000.0,tt2527336,2017-12-13,1.332540e+09,Star Wars: The Last Jedi,12,2017,PG-13,Holiday,Science Fiction,1.132540e+09


In [46]:
clean_movie_data.to_csv('clean_movie_data.csv', index = False)

# Movie Actor Data

- Merge IMDB dataframes related to movie actos
- Specify actors in movies released in the US
- Drop unnecessary columns and duplicate values
- Merge cleaned dataframe with movie_data

In [24]:
#joined these tables to get the name and the specific profession ('category') they had for that movie

name_p = imdb_name.merge(imdb_title_p, on = 'nconst', how = 'left') 
name_p.head()

,nconst,primary_name,birth_year,death_year,primary_profession,known_for_titles,tconst,ordering,category,job,characters
0,nm0061671,Mary Ellen Bauder,NaN,NaN,"miscellaneous,production_manager,producer","tt0837562,tt2398241,tt0844471,tt0118553",tt2398241,9.0,producer,producer,NaN
1,nm0061865,Joseph Bauer,NaN,NaN,"composer,music_department,sound_department","tt0896534,tt6791238,tt0287072,tt1682940",tt0433397,7.0,composer,NaN,NaN
2,nm0061865,Joseph Bauer,NaN,NaN,"composer,music_department,sound_department","tt0896534,tt6791238,tt0287072,tt1682940",tt1681372,8.0,composer,NaN,NaN
3,nm0061865,Joseph Bauer,NaN,NaN,"composer,music_department,sound_department","tt0896534,tt6791238,tt0287072,tt1682940",tt2387710,8.0,composer,NaN,NaN
4,nm0061865,Joseph Bauer,NaN,NaN,"composer,music_department,sound_department","tt0896534,tt6791238,tt0287072,tt1682940",tt2281215,7.0,composer,NaN,NaN


In [25]:
#renamed the 'tconst' column to match the title_id column and use that later to join with title_ba table
name_p = name_p.rename(columns = {"tconst": "title_id", "nconst": "name_id"}) 

In [26]:
#joined both tables to get the name and the title info together
name_titles = title_ba.merge(name_p, on = 'title_id', how = 'left')
name_titles.head()

,title_id,primary_title,original_title,start_year,runtime_minutes,genres,ordering_x,title,region,language,...,name_id,primary_name,birth_year,death_year,primary_profession,known_for_titles,ordering_y,category,job,characters
0,tt0063540,Sunghursh,Sunghursh,2013,175.0,"Action,Crime,Drama",1.0,Sangharsh,IN,hi,...,nm0347899,Gulzar,1936.0,NaN,"music_department,writer,soundtrack","tt0091256,tt0178186,tt1010048,tt2176013",8.0,writer,dialogue,NaN
1,tt0063540,Sunghursh,Sunghursh,2013,175.0,"Action,Crime,Drama",1.0,Sangharsh,IN,hi,...,nm1194313,Mahasweta Devi,1926.0,2016.0,writer,"tt0108001,tt0832902,tt0063540,tt0178562",7.0,writer,story,NaN
2,tt0063540,Sunghursh,Sunghursh,2013,175.0,"Action,Crime,Drama",1.0,Sangharsh,IN,hi,...,nm0712540,Harnam Singh Rawail,1921.0,2004.0,"director,writer,producer","tt0266757,tt0266712,tt0134996,tt0266765",5.0,director,NaN,NaN
3,tt0063540,Sunghursh,Sunghursh,2013,175.0,"Action,Crime,Drama",1.0,Sangharsh,IN,hi,...,nm0904537,Vyjayanthimala,1933.0,NaN,"actress,music_department,miscellaneous","tt0047990,tt0054910,tt0050665,tt0058547",2.0,actress,NaN,"[""Munni"",""Laila-E-Aasmaan""]"
4,tt0063540,Sunghursh,Sunghursh,2013,175.0,"Action,Crime,Drama",1.0,Sangharsh,IN,hi,...,nm0023551,Abrar Alvi,1927.0,2009.0,"writer,actor,director","tt0071811,tt0359496,tt0056436,tt0061046",6.0,writer,dialogue,NaN


In [27]:
#grouped by region and used get group to only show the US movies
us_name_titles = name_titles.groupby('region').get_group('US')
us_name_titles.head()

,title_id,primary_title,original_title,start_year,runtime_minutes,genres,ordering_x,title,region,language,...,name_id,primary_name,birth_year,death_year,primary_profession,known_for_titles,ordering_y,category,job,characters
128,tt0069049,The Other Side of the Wind,The Other Side of the Wind,2018,122.0,Drama,2.0,The Other Side of the Wind,US,NaN,...,nm0006166,Michel Legrand,1932.0,2019.0,"composer,soundtrack,music_department","tt0067334,tt0067803,tt0086619,tt0063688",8.0,composer,NaN,NaN
129,tt0069049,The Other Side of the Wind,The Other Side of the Wind,2018,122.0,Drama,2.0,The Other Side of the Wind,US,NaN,...,nm0462648,Oja Kodar,1941.0,NaN,"actress,writer,director","tt0069049,tt0108506,tt0209994,tt0239070",2.0,actress,NaN,"[""The Actress""]"
130,tt0069049,The Other Side of the Wind,The Other Side of the Wind,2018,122.0,Drama,2.0,The Other Side of the Wind,US,NaN,...,nm0004372,Gary Graver,1938.0,2006.0,"cinematographer,director,camera_department","tt0178845,tt0121549,tt0207568,tt0082004",9.0,cinematographer,NaN,NaN
131,tt0069049,The Other Side of the Wind,The Other Side of the Wind,2018,122.0,Drama,2.0,The Other Side of the Wind,US,NaN,...,nm0001379,John Huston,1906.0,1987.0,"actor,director,writer","tt0040897,tt0040506,tt0071315,tt0043265",1.0,actor,NaN,"[""Jake Hannaford""]"
132,tt0069049,The Other Side of the Wind,The Other Side of the Wind,2018,122.0,Drama,2.0,The Other Side of the Wind,US,NaN,...,nm0613657,Bob Murawski,1964.0,NaN,"editor,producer,editorial_department","tt0887912,tt0316654,tt0145487,tt0069049",10.0,editor,NaN,NaN


In [28]:
#check columns
us_name_titles.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 388927 entries, 128 to 2975188
Data columns (total 23 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   title_id            388927 non-null  object 
 1   primary_title       388927 non-null  object 
 2   original_title      388926 non-null  object 
 3   start_year          388927 non-null  int64  
 4   runtime_minutes     331117 non-null  float64
 5   genres              385776 non-null  object 
 6   ordering_x          388927 non-null  float64
 7   title               388927 non-null  object 
 8   region              388927 non-null  object 
 9   language            5128 non-null    object 
 10  types               62174 non-null   object 
 11  attributes          22294 non-null   object 
 12  is_original_title   388922 non-null  float64
 13  name_id             388634 non-null  object 
 14  primary_name        388634 non-null  object 
 15  birth_year          100819 non-

In [29]:
#drop irrelevant colunms
clean_us_names = us_name_titles.drop(columns = ['primary_title', 'original_title', 'start_year', 'language', 'types', 'attributes', 'is_original_title', 'birth_year', 'death_year', 'genres', 'ordering_x', 'ordering_y', 'primary_profession', 'known_for_titles', 'job'])
clean_us_names.head()

,title_id,runtime_minutes,title,region,name_id,primary_name,category,characters
128,tt0069049,122.0,The Other Side of the Wind,US,nm0006166,Michel Legrand,composer,NaN
129,tt0069049,122.0,The Other Side of the Wind,US,nm0462648,Oja Kodar,actress,"[""The Actress""]"
130,tt0069049,122.0,The Other Side of the Wind,US,nm0004372,Gary Graver,cinematographer,NaN
131,tt0069049,122.0,The Other Side of the Wind,US,nm0001379,John Huston,actor,"[""Jake Hannaford""]"
132,tt0069049,122.0,The Other Side of the Wind,US,nm0613657,Bob Murawski,editor,NaN


In [30]:
#filter to just get actor and actress names per movie
clean_us_actors = clean_us_names[clean_us_names['category'].isin(['actor', 'actress'])]

#drop any duplicates that have the same title_id and name_id
clean_us_actors = clean_us_actors.drop_duplicates(subset=(['title_id', 'name_id']), keep="first")

In [31]:
#rename columns to match up with movie_data for merging
clean_us_actors = clean_us_actors.rename(columns = {"title_id": "imdb_id", "primary_name": "name"})
clean_us_actors.head()

,imdb_id,runtime_minutes,title,region,name_id,name,category,characters
129,tt0069049,122.0,The Other Side of the Wind,US,nm0462648,Oja Kodar,actress,"[""The Actress""]"
131,tt0069049,122.0,The Other Side of the Wind,US,nm0001379,John Huston,actor,"[""Jake Hannaford""]"
133,tt0069049,122.0,The Other Side of the Wind,US,nm0000953,Peter Bogdanovich,actor,"[""Brooks Otterlake""]"
136,tt0069049,122.0,The Other Side of the Wind,US,nm0001782,Susan Strasberg,actress,"[""Julie Rich""]"
488,tt0159369,180.0,Cooper and Hemingway: The True Gen,US,nm0186908,Nancy Crawford,actress,"[""Additional voices""]"


In [32]:
#merge movie_data with clean_us_actors on imdb_id column
movie_actor = clean_movie_data.merge(clean_us_actors, on = 'imdb_id', how = 'left')

In [33]:
#drop any remaining columns we don't need for analysis
movie_actor = movie_actor.drop(columns = ['runtime_minutes', 'title_y', 'region', 'imdb_id', 'name_id'])

In [34]:
#rename title
movie_actor = movie_actor.rename(columns = {"title_x":"title"})

In [35]:
movie_actor.head()

,budget,release_date,revenue,title,month,year,rating,season,genre,profitability,name,category,characters
0,356000000.0,2019-04-24,2.797801e+09,Avengers: Endgame,4,2019,PG-13,Spring,Adventure,2.441801e+09,Mark Ruffalo,actor,"[""Bruce Banner"",""Hulk""]"
1,356000000.0,2019-04-24,2.797801e+09,Avengers: Endgame,4,2019,PG-13,Spring,Adventure,2.441801e+09,Chris Hemsworth,actor,"[""Thor""]"
2,356000000.0,2019-04-24,2.797801e+09,Avengers: Endgame,4,2019,PG-13,Spring,Adventure,2.441801e+09,Chris Evans,actor,"[""Steve Rogers"",""Captain America""]"
3,356000000.0,2019-04-24,2.797801e+09,Avengers: Endgame,4,2019,PG-13,Spring,Adventure,2.441801e+09,Robert Downey Jr.,actor,"[""Tony Stark"",""Iron Man""]"
4,300000000.0,2018-04-25,2.046240e+09,Avengers: Infinity War,4,2018,PG-13,Spring,Adventure,1.746240e+09,Mark Ruffalo,actor,"[""Bruce Banner"",""Hulk""]"


In [45]:
movie_actor.to_csv('movie_actor.csv', index = False)

### Additional Steps taken before visualization

- below are the lines of code needed to get the correct dataframe for visualization
- broadly, we needed to drop additional columns, add a profitibility column, adjust the float format, and create a few variables

In [ ]:
movies_df = pd.read_csv('movies_data.csv')

In [ ]:
month_movies_df = movies_df.sort_values(by=['month'], ascending=[True])
month_movies_df

In [ ]:
test_df = movies_df.drop(columns=['genre_7', 'genre_6', 'genre_5', 'genre_4', 'genre_3', 'genre_2'])
test_df = test_df.sort_values(by=['revenue'], ascending=[False])
rating_test_df = test_df.dropna(subset=['rating'])
rating_test_df

In [ ]:
clean_rating_df = rating_test_df[rating_test_df['rating'].isin(['G', 'PG', 'PG-13', 'R'])]

In [ ]:
clean_rating_df['profitability'] = clean_rating_df['revenue'] - clean_rating_df['budget']

In [ ]:
clean_budget_df = clean_rating_df.loc[~(clean_rating_df['budget'] == 0)]
clean_genre_df = clean_budget_df.dropna(subset=['genre_1'])
clean_genre_df

In [ ]:
corr = clean_budget_df.corr()

In [ ]:
clean_genre_df.groupby('genre_1')['profitability'].mean()

In [ ]:
pd.options.display.float_format = "{:.2f}".format

clean_genre_df.describe()

In [ ]:
clean_genre_df.head()

In [ ]:
last_ten_years = clean_genre_df.loc[clean_genre_df['year']>=2010]
last_ten_years

In [ ]:
movies_by_revenue = last_ten_years['revenue'].mean()
movies_by_revenue

In [ ]:
clean_genre_df.to_csv('clean_movie_data_final.csv')